In [9]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [10]:
df = pd.read_csv("D:\Sanjay\github\Assestments\Instagram-Reach-Prediction\Dataset\instagram_reach.csv")
df.head(2)

,Unnamed: 0,S.No,USERNAME,Caption,Followers,Hashtags,Time since posted,Likes
0,0,1,mikequindazzi,Who are #DataScientist and what do they do? >>...,1600,#MachineLearning #AI #DataAnalytics #DataScien...,11 hours,139
1,1,2,drgorillapaints,We all know where it’s going. We just have to ...,880,#deck .#mac #macintosh#sayhello #apple #steve...,2 hours,23


In [11]:
df['Time since posted'] = df['Time since posted'].str.replace(" hours","")


In [12]:
df['Time since posted']= df['Time since posted'].astype('int64')


In [13]:
df =df.drop(['S.No','USERNAME',"Caption","Unnamed: 0","Hashtags"], axis=1)


# preparing X and Y

In [14]:
X= df.drop(columns=['Likes','Time since posted'],axis=1)
y1 = df['Likes']
y2 = df['Time since posted']

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X,y1,y2,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y1_train.shape, y1_test.shape, y1_train.shape, y1_test.shape

((80, 1), (20, 1), (80,), (20,), (80,), (20,))

In [16]:
X_train

,Followers
55,3448
88,1158
26,265
42,273
69,1003
...,...
60,145
71,383
14,2904
92,106


In [17]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [18]:
y_train = np.column_stack((y1_train, y2_train))
y_test = np.column_stack((y1_test, y2_test))

In [19]:
params={
                "Decision Tree": {
                    'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                    # 'splitter':['best','random'],
                    # 'max_features':['sqrt','log2'],
                },
                "Random Forest Regressor":{
                    'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                 
                    # 'max_features':['sqrt','log2',None],
                    'n_estimators': [8,16,32,64,128,115,256]
                },
                "Gradient Boosting":{
                    # 'loss':['squared_error', 'huber', 'absolute_error', 'quantile'],
                    'learning_rate':[.1,.01,.05,.001],
                    'subsample':[0.6,0.7,0.75,0.8,0.85,0.9],
                    # 'criterion':['squared_error', 'friedman_mse'],
                    # 'max_features':['auto','sqrt','log2'],
                    'n_estimators': [8,16,32,64,128,256]
                },
                "Linear Regression":{},
                "XGBRegressor":{
                    'learning_rate':[.1,.01,.05,.001],
                    'n_estimators': [8,16,32,64,128,256]
                },
                "CatBoosting Regressor":{
                    'depth': [6,8,10],
                    'learning_rate': [0.01, 0.05, 0.1],
                    'iterations': [30, 50, 100]
                },
                "AdaBoost Regressor":{
                    'learning_rate':[.1,.01,0.5,.001],
                    # 'loss':['linear','square','exponential'],
                    'n_estimators': [8,16,32,64,128,256]
                },
                "Lasso":{},
                "Ridge":{}
                
            }        

In [20]:
list(models.keys())[1]

NameError: name 'models' is not defined

In [26]:
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
}
#model_list = []
#r2_list =[]
model_performance = dict()

for i in range(len(list(models))):
    model = list(models.values())[i]
    print(model,list(models.keys())[i])
    para =  params[list(models.keys())[i]]
    print(model, para)
    y_train = np.column_stack((y1_train, y2_train))
    y_test = np.column_stack((y1_test, y2_test))
    
    gs = GridSearchCV(model,para,cv=3)
    gs.fit(X_train,y_train)
    model.set_params(**gs.best_params_)
    model.fit(X_train, y_train) # Train model
    


    # Make predictions
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    y1_train_pred, y2_train_pred =np.hsplit(y_train_pred, 2)
    y1_test_pred, y2_test_pred = np.hsplit( y_test_pred, 2)
    
    # Evaluate Train and Test dataset
    model_train_mae_y1 , model_train_rmse_y1, model_train_r2_y1 = evaluate_model(y1_train, y1_train_pred)
    model_train_mae_y2 , model_train_rmse_y2, model_train_r2_y2 = evaluate_model(y2_train, y2_train_pred)

    model_test_mae_y1 , model_test_rmse_y1, model_test_r2_y1 = evaluate_model(y1_test, y1_test_pred)
    model_test_mae_y2 , model_test_rmse_y2, model_test_r2_y2 = evaluate_model(y2_test, y2_test_pred)

    
    print(list(models.keys())[i])
    #model_list.append(list(models.keys())[i])
    model_performance[list(models.keys())[i]]= dict()
    
    print('Model performance for Training set y1')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse_y1))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae_y1))
    print("- R2 Score: {:.4f}".format(model_train_r2_y1))
    
    print('Model performance for Training set y2')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse_y2))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae_y2))
    print("- R2 Score: {:.4f}".format(model_train_r2_y2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse_y1))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae_y1))
    print("- R2 Score: {:.4f}".format(model_test_r2_y1))
    
    print('Model performance for Test set y2')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse_y2))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae_y2))
    print("- R2 Score: {:.4f}".format(model_test_r2_y2))
    #r2_list.append(model_test_r2)
    model_performance[list(models.keys())[i]]["y1"] = model_test_r2_y1
    model_performance[list(models.keys())[i]]["y2"] = model_test_r2_y2
    
    
    print('='*35)
    print('\n')

LinearRegression() Linear Regression
LinearRegression() {}
Linear Regression
Model performance for Training set y1
- Root Mean Squared Error: 56.5118
- Mean Absolute Error: 33.5100
- R2 Score: 0.0595
Model performance for Training set y2
- Root Mean Squared Error: 3.2489
- Mean Absolute Error: 1.6515
- R2 Score: 0.0637
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 34.4816
- Mean Absolute Error: 25.5902
- R2 Score: 0.1613
Model performance for Test set y2
- Root Mean Squared Error: 3.3768
- Mean Absolute Error: 2.1777
- R2 Score: 0.0210


DecisionTreeRegressor() Decision Tree
DecisionTreeRegressor() {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']}
Decision Tree
Model performance for Training set y1
- Root Mean Squared Error: 2.1809
- Mean Absolute Error: 0.5875
- R2 Score: 0.9986
Model performance for Training set y2
- Root Mean Squared Error: 0.3708
- Mean Absolute Error: 0.1000
- R2 Score: 0.9878
-------------

KeyboardInterrupt: 

In [27]:
no_cv=model_performance
no_cv

{'Linear Regression': {'y1': 0.16128913504989728, 'y2': 0.021029172852261024},
 'Decision Tree': {'y1': 0.8695355444219304, 'y2': 0.12856836230950863},
 'Random Forest Regressor': {'y1': 0.8158461925039469,
  'y2': 0.5125883892585439}}

In [28]:
## Selecting Random Forest
rf_model =  RandomForestRegressor()

In [29]:
grid_param = {
    "n_estimators" : [90,100,115,130],
}

In [30]:
# hyperparementer tuning using grid search cv 
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator = rf_model, param_grid = grid_param, cv =3, verbose =1, n_jobs=-1)

In [31]:
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'n_estimators': [90, 100, 115, 130]}, verbose=1)

In [32]:
param=grid_search.best_params_

In [33]:
rf_model.set_params(**grid_search.best_params_)

RandomForestRegressor()

In [34]:
rf_model.fit(X_train,y_train)

RandomForestRegressor()

In [35]:
p=rf_model.predict(X_test)

In [37]:
p

array([[ 16.92      ,   2.04      ],
       [ 16.92      ,   2.04      ],
       [ 22.03      ,   2.04      ],
       [ 19.78      ,   2.        ],
       [ 19.78      ,   2.        ],
       [112.77      ,  14.86      ],
       [ 43.97416667,   2.18916667],
       [ 21.07      ,   2.        ],
       [ 25.85      ,   2.        ],
       [112.77      ,  14.86      ],
       [ 16.67      ,   2.77      ],
       [ 70.48      ,   2.        ],
       [ 61.75      ,   3.74      ],
       [ 36.        ,   3.51      ],
       [ 28.62      ,   2.        ],
       [ 32.02985714,   3.63661905],
       [ 14.27      ,   2.51      ],
       [ 36.        ,   3.51      ],
       [ 44.06416667,   2.18916667],
       [ 13.81      ,   3.16      ]])

In [113]:
y1_train_pred, y2_train_pred =np.hsplit(p, 2)
y1_test_pred, y2_test_pred = np.hsplit( p, 2)

In [114]:
r2_score(y2_test,y2_test_pred)

0.6091035453483105

In [115]:
r2_score(y1_test,y1_test_pred)

0.7458764865241398

In [117]:
y1_test_pred

array([[ 15.36 ],
       [ 15.36 ],
       [ 23.25 ],
       [ 20.07 ],
       [ 19.95 ],
       [105.18 ],
       [ 51.761],
       [ 20.28 ],
       [ 25.53 ],
       [105.18 ],
       [ 16.8  ],
       [ 70.6  ],
       [ 54.41 ],
       [ 34.94 ],
       [ 27.52 ],
       [ 31.98 ],
       [ 13.29 ],
       [ 34.83 ],
       [ 51.851],
       [ 14.44 ]])

In [98]:
i = pd.DataFrame({"Features":[800]})
i

,Features
0,800


In [94]:
b=np.array([880])
b

array([880])

In [99]:
b.reshape(1,-1)

array([[880]])

In [95]:
a= rf_model.predict(b.reshape(1,-1))

C:\Users\sanjay\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [96]:
np.hsplit(a, 2)

[array([[27.73]]), array([[2.]])]

In [91]:
a

array([[27.73]])

In [92]:
b

array([[2.]])